In [1]:
import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import os

In [2]:
train = pd.read_csv('./data/boneage-training-dataset.csv')
test = pd.read_csv('./data/boneage-test-dataset.csv', sep=';')

In [3]:
train_path = './data/boneage-training-dataset/boneage-training-dataset/post'
train_size = 10000
X_train = []
y_train = []

for img in os.listdir(train_path):
    if train.loc[lambda df: df['id'] == int(img.split('.')[0])].male.values[0]: 
        if len(X_train) == train_size:
            break
        image = Image.open(train_path+"/"+img)
        image = ImageOps.grayscale(image)
        image = image.resize((128,128))
        image = np.array(image)
        image = image.reshape((128, 128, 1))
        X_train.append(image / 255)
        y_train.append(train.loc[lambda df: df['id'] == int(img.split('.')[0])].boneage.values[0])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [4]:
test_path = './data/boneage-test-dataset/boneage-test-dataset/post'
X_test = []
y_test = []

for img in os.listdir(test_path):
    if test.loc[lambda df: df['id'] == int(img.split('.')[0])].male.values[0]: 
        image = Image.open(test_path+"/"+img)
        image = ImageOps.grayscale(image)
        image = image.resize((128,128))
        image = np.array(image)
        image = image.reshape((128, 128, 1))
        X_test.append(image / 255)
        y_test.append(test.loc[lambda df: df['id'] == int(img.split('.')[0])].boneage.values[0])

X_test = np.array(X_test)
y_test = np.array(y_test)

In [5]:
model = models.Sequential()
# model.add(layers.InputLayer((512, 512, 1)))
model.add(layers.Conv2D(4, (8, 8), activation='relu', input_shape=(128, 128, 1)))
model.add(layers.Dropout(0.25))
# model.add(layers.AveragePooling2D((2,2)))
model.add(layers.Conv2D(4, (8, 8), activation='relu'))
model.add(layers.Dropout(0.25))
# model.add(layers.AveragePooling2D((2,2)))
# model.add(layers.Conv2D(32, (2,2), activation='relu'))
# model.add(layers.AveragePooling2D((2,2)))

In [6]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 121, 121, 4)       260       
                                                                 
 dropout (Dropout)           (None, 121, 121, 4)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 114, 114, 4)       1028      
                                                                 
 dropout_1 (Dropout)         (None, 114, 114, 4)       0         
                                                                 
 flatten (Flatten)           (None, 51984)             0         
                                                                 
 dense (Dense)               (None, 128)               6654080   
                                                                 
 dense_1 (Dense)             (None, 1)                 1

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.mean_squared_error,
              metrics=['mean_absolute_error'])

In [9]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
35/35 [==============================] - 17s 456ms/step - loss: 5977.8110 - mean_absolute_error: 58.6351 - val_loss: 4187.9814 - val_mean_absolute_error: 59.0292
Epoch 2/100
35/35 [==============================] - 18s 516ms/step - loss: 1760.2148 - mean_absolute_error: 33.2074 - val_loss: 2921.8936 - val_mean_absolute_error: 47.8315
Epoch 3/100
35/35 [==============================] - 18s 507ms/step - loss: 1669.6836 - mean_absolute_error: 32.0138 - val_loss: 2818.5798 - val_mean_absolute_error: 47.2134
Epoch 4/100
35/35 [==============================] - 18s 507ms/step - loss: 1625.0970 - mean_absolute_error: 31.4499 - val_loss: 3969.4141 - val_mean_absolute_error: 57.5446
Epoch 5/100
35/35 [==============================] - 18s 507ms/step - loss: 1667.5978 - mean_absolute_error: 32.3832 - val_loss: 2773.2014 - val_mean_absolute_error: 46.7079
Epoch 6/100
35/35 [==============================] - 18s 507ms/step - loss: 1569.6218 - mean_absolute_error: 31.0907 - val_loss: 2

KeyboardInterrupt: 

In [10]:
pd.DataFrame({'prediction': model.predict(X_test).tolist(), 'boneage': y_test.tolist()})

,prediction,boneage
0,[121.4689712524414],161
1,[121.141845703125],135
2,[120.48489379882812],136
3,[119.61840057373047],87
4,[122.13145446777344],143
5,[119.49249267578125],143
6,[125.31494140625],199
7,[118.38359069824219],150
8,[121.18272399902344],166
9,[120.83247375488281],155
